In [3]:
# !huggingface-cli login

In [11]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /mnt/ceph/storage/data-tmp/teaching-current//aq60qovu/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [28]:
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split

from transformers import (
    #AutoTokenizer,
    DistilBertTokenizerFast,
    TFAutoModelForSequenceClassification,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
    PushToHubCallback
)

In [22]:
def load_data():
    print(f'Loading data...')
    # Load data as series
    _X_train = \
        pd.read_csv(
            '../../datasets/future_statements_dataset/X_train.csv'
        )['statement']
    _y_train = \
        pd.read_csv(
            '../../datasets/future_statements_dataset/y_train.csv'
        )['future']

    # Create train/test split
    _X_train, _X_test, _y_train, _y_test = \
        train_test_split(_X_train, _y_train, test_size=0.2)

    # Create train/validation split
    _X_train, _X_valid, _y_train, _y_valid = \
        train_test_split(_X_train, _y_train, test_size=0.2)

    # Sort index
    _X_train.sort_index(inplace=True)
    _X_valid.sort_index(inplace=True)
    _X_test.sort_index(inplace=True)
    _y_train.sort_index(inplace=True)
    _y_valid.sort_index(inplace=True)
    _y_test.sort_index(inplace=True)

    # Reset index
    _X_train.reset_index(drop=True, inplace=True)
    _X_valid.reset_index(drop=True, inplace=True)
    _X_test.reset_index(drop=True, inplace=True)
    _y_train.reset_index(drop=True, inplace=True)
    _y_valid.reset_index(drop=True, inplace=True)
    _y_test.reset_index(drop=True, inplace=True)

    print('Training data:   ', len(_X_train.index), ' rows.')
    print('Validation data: ', len(_X_valid.index), ' rows.')
    print('Test data:       ', len(_X_test.index), ' rows.')

    print(f'Loading done...')
    return _X_train.tolist(), _X_valid.tolist(), _X_test.tolist(), \
        _y_train.tolist(), _y_valid.tolist(), _y_test.tolist()

def encode_data(dataset, _tokenizer):
    print(f'Encoding data...')
    inputs = _tokenizer(dataset,
                        max_length=128,
                        padding='max_length',
                        truncation=True,
                        return_token_type_ids=False)
    ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    print(f'Encoding done...')
    return tf.convert_to_tensor(ids), tf.convert_to_tensor(attention_mask)

In [23]:
X_train, X_valid, X_test, y_train, y_valid, y_test = load_data()

Loading data...
Training data:    1600  rows.
Validation data:  400  rows.
Test data:        500  rows.
Loading done...


In [24]:
model = TFAutoModelForSequenceClassification.from_pretrained(
        'distilbert-base-uncased',
        num_labels=2
    )
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)
tokenizer = DistilBertTokenizerFast.from_pretrained(
    'distilbert-base-uncased'
)

2022-08-17 00:31:06.242344: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-17 00:31:06.370524: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-17 00:31:06.370676: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (betaweb036): /proc/driver/nvidia/version does not exist
Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_transform', 'vocab_projector', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTra

In [25]:
# Get train_ids and attention_mask
X_train_ids, X_train_attention_mask = encode_data(X_train, tokenizer)
X_valid_ids, X_valid_attention_mask = encode_data(X_valid, tokenizer)
X_test_ids, X_test_attention_mask = encode_data(X_test, tokenizer)

Encoding data...
Encoding done...
Encoding data...
Encoding done...
Encoding data...
Encoding done...


In [29]:
# Callback
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                  mode='min',
                                                  min_delta=0,
                                                  patience=0,
                                                  restore_best_weights=True)

In [31]:
push_cb = [PushToHubCallback("model_output/", 
                               tokenizer=tokenizer,
                               hub_model_id="bert-fine-tuned-cola")]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/mnt/ceph/storage/data-tmp/teaching-current/aq60qovu/the-future-tense/src/future_model_ft/model_output is already a clone of https://huggingface.co/fidsinn/bert-fine-tuned-cola. Make sure you pull the latest changes with `repo.git_pull()`.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [ ]:
train_history = model.fit(
    x=[X_train_ids, X_train_attention_mask],
    y=np.asarray(y_train),
    epochs=6,
    batch_size=64,
    steps_per_epoch=len(X_train) // 64,
    validation_data=([X_valid_ids, X_valid_attention_mask],
                     np.asarray(y_valid)),
    callbacks=[early_stopping,
              push_cb
              ],
    verbose=1
)

Epoch 1/6


2022-08-17 00:36:13.657920: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 100663296 exceeds 10% of free system memory.
2022-08-17 00:36:14.373703: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 100663296 exceeds 10% of free system memory.
2022-08-17 00:36:14.373822: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 100663296 exceeds 10% of free system memory.
2022-08-17 00:36:14.466509: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 100663296 exceeds 10% of free system memory.
2022-08-17 00:36:14.466608: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 100663296 exceeds 10% of free system memory.


 7/25 [=======>......................] - ETA: 13:12 - loss: 0.6772 - sparse_categorical_accuracy: 0.6629

In [ ]:
model.push_to_hub("distilbert-base-future", commit_message="End of training")